In [ ]:
# import library ที่จำเป็น
import pandas as pd
import seaborn as sns
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from IPython.display import clear_output
from torch.utils.data import Dataset, DataLoader

In [ ]:
# ฟังก์ชั่นสำหรับทำ visualization ดูการกระจายตัวของ y predict & y true

#Data visualization
import mpl_scatter_density # adds projection='scatter_density'
from matplotlib.colors import LinearSegmentedColormap

# "Viridis-like" colormap with white background
white_viridis = LinearSegmentedColormap.from_list('white_viridis', [
    (0, '#ffffff'),
    (1e-20, '#440053'),
    (0.01, '#404388'),
    (0.05, '#2a788e'),
    (0.1, '#21a784'),
    (0.2, '#78d151'),
    (1, '#fde624'),
], N=y_test.shape[0])

def using_mpl_scatter_density(fig, x, y):
    ax = fig.add_subplot(1, 1, 1, projection='scatter_density')
    density = ax.scatter_density(x, y, cmap=white_viridis)
    fig.colorbar(density, label='Number of points per pixel')
    plt.plot([-50.0,800.0], [-50.0,800.0], "--", color = "Black")
    plt.ylabel('Predicted WC')
    plt.xlabel('Real data WC')

In [ ]:
# นิยามตัวแปร device สำหรับการใช้งานใน gpu หาก available ถ้าไม่มีใช้ cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# นิยามสำหรับ categorical value เป็น index

REPLACE_CATS = {
    "metal_linker":[1, 2, 3, 4, 9, 10, 12], 
    'organic_linker1':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 
    'organic_linker2':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 
    'functional_groups':['Br-NHMe', 'I-OH', 'Br-OPr', 'Et-H', 'OPr', 'NO2-Me', 'SO3H-NH2', 'NH2-F', 'NO2-Pr', 'Br-Pr', 'Ph-Et', 'OH', 'OEt-Et', 'Pr-Et', 'OH-SO3H', 'Et-Me', 'OH-Et', 'I-F', 'Pr-COOH', 'SO3H-F', 'Ph-OH', 'OH-OMe', 'H-Pr', 'COOH-Me', 'COOH-F', 'Et-SO3H', 'COOH-OMe', 'Me-NHMe', 'F-CN', 'NO2-Ph', 'Pr-F', 'CN-NO2', 'H-OH', 'Me-HCO', 'Ph-Cl', 'COOH-OPr', 'OH-HCO', 'Pr-OEt', 'OPr-Et', 'OH-Pr', 'Cl-OH', 'OMe-COOH', 'OMe-H', 'SO3H-NHMe', 'H-NO2', 'OPr-OEt', ' ', 'NO2-NH2', 'OPr-CN', 'I-Ph', 'H-Et', 'NH2-OMe', 'OH-H', 'Me-SO3H', 'OEt-H', 'Pr-I', 'F-Et', 'COOH-NHMe', 'HCO-Me', 'CN-NHMe', 'NO2-CN', 'Pr-OPr', 'CN-HCO', 'Br-COOH', 'OPr-Br', 'OH-Ph', 'OEt-SO3H', 'Cl-SO3H', 'NO2-HCO', 'Cl-CN', 'Pr-Me', 'NH2-NHMe', 'Pr-HCO', 'OEt-NHMe', 'NHMe-Pr', 'I-HCO', 'OEt-CN', 'Ph-COOH', 'Cl-Me', 'Et-NH2', 'Et', 'Ph-SO3H', 'F', 'Ph-CN', 'OEt-OMe', 'F-NH2', 'F-SO3H', 'NO2-I', 'HCO-Br', 'OEt-HCO', 'NHMe-Me', 'HCO-CN', 'Br-H', 'CN', 'Cl-OEt', 'HCO-COOH', 'Et-I', 'H-COOH', 'F-OEt', 'Pr-SO3H', 'COOH-CN', 'Et-OH', 'H-NHMe', 'SO3H-OH', 'Pr-Cl', 'OEt-OPr', 'Me-Et', 'Ph-H', 'SO3H-CN', 'Ph-NH2', 'Br-Me', 'Et-OPr', 'CN-H', 'NH2-HCO', 'Et-OMe', 'I-Br', 'Ph-OPr', 'H-Ph', 'NHMe-NO2', 'Et-Ph', 'I-CN', 'COOH-H', 'CN-I', 'COOH-OH', 'Et-Br', 'I-NO2', 'OEt-NH2', 'SO3H-Cl', 'COOH-NO2', 'SO3H-OEt', 'Me', 'OPr-COOH', 'H-SO3H', 'OPr-Cl', 'SO3H-Ph', 'CN-OH', 'Cl-Ph', 'Br-OMe', 'Me-CN', 'OH-Me', 'SO3H-Me', 'Me-OH', 'CN-COOH', 'F-NO2', 'Ph-OMe', 'NHMe-COOH', 'CN-OEt', 'OEt-Me', 'OMe-Cl', 'OMe-NO2', 'F-OPr', 'NHMe-Cl', 'NH2-H', 'H-CN', 'OMe-SO3H', 'Ph-F', 'F-H', 'F-Cl', 'OEt-Br', 'F-Br', 'SO3H-NO2', 'HCO-NH2', 'HCO-NHMe', 'SO3H', 'NO2-NHMe', 'NH2-NO2', 'NO2-OPr', 'NH2-Ph', 'H-Cl', 'Ph', 'H-NH2', 'Br', 'Ph-OEt', 'Cl-Pr', 'I-Et', 'OH-Br', 'NO2-SO3H', 'NHMe-HCO', 'Cl-I', 'Pr-OMe', 'NH2-OEt', 'OPr-Me', 'NO2-Et', 'OMe-Pr', 'Me-NH2', 'Br-OH', 'H-OMe', 'Me-OEt', 'SO3H-Et', 'Et-Pr', 'OPr-NO2', 'OMe-I', 'Cl-Br', 'NO2-OMe', 'CN-NH2', 'Ph-NO2', 'Pr-Br', 'NO2', 'Et-COOH', 'Pr-Ph', 'HCO-H', 'OPr-H', 'Ph-Me', 'HCO-NO2', 'NHMe-Br', 'NHMe-NH2', 'OPr-SO3H', 'Br-NO2', 'I-Pr', 'NHMe-H', 'F-Ph', 'Cl-OPr', 'HCO-OMe', 'COOH-Br', 'OEt-OH', 'OPr-I', 'NH2-Me', 'Me-COOH', 'H-OPr', 'SO3H-OMe', 'OMe-OH', 'Me-NO2', 'HCO-OPr', 'NHMe-OPr', 'OH-OPr', 'COOH-NH2', 'Ph-NHMe', 'HCO', 'NHMe', 'SO3H-Br', 'CN-Et', 'F-OH', 'I', 'HCO-Ph', 'NHMe-SO3H', 'NO2-OEt', 'OH-NHMe', 'COOH-HCO', 'Cl-NH2', 'NHMe-CN', 'F-NHMe', 'I-SO3H', 'Me-OPr', 'SO3H-I', 'Cl-OMe', 'NH2-Pr', 'Pr-OH', 'COOH-OEt', 'NH2', 'H-I', 'Cl', 'NO2-F', 'Cl-F', 'NH2-Cl', 'Br-OEt', 'Pr', 'Ph-I', 'OH-OEt', 'OMe-CN', 'OMe-NHMe', 'Me-I', 'NH2-SO3H', 'OMe-F', 'NHMe-I', 'COOH', 'OH-COOH', 'SO3H-HCO', 'Et-HCO', 'NH2-COOH', 'Br-CN', 'I-Cl', 'CN-Ph', 'I-NHMe', 'Br-SO3H', 'NO2-H', 'OMe-Ph', 'H-HCO', 'NH2-OH', 'Cl-HCO', 'OMe-Me', 'I-OMe', 'CN-OPr', 'Pr-CN', 'F-OMe', 'HCO-OH', 'HCO-Cl', 'Cl-COOH', 'OEt-I', 'Br-Et', 'OMe-OPr', 'F-Pr', 'Me-F', 'COOH-SO3H', 'NHMe-Ph', 'NHMe-OH', 'CN-Me', 'Pr-H', 'HCO-SO3H', 'Me-Br', 'F-COOH', 'I-OPr', 'OPr-HCO', 'NHMe-Et', 'I-COOH', 'HCO-I', 'OPr-OMe', 'OMe-Br', 'NH2-CN', 'OMe-NH2', 'CN-F', 'OEt-F', 'F-HCO', 'H-OEt', 'OH-NO2', 'Cl-NHMe', 'OMe-HCO', 'OPr-Ph', 'OPr-F', 'NH2-OPr', 'OEt-Ph', 'HCO-Pr', 'Br-HCO', 'Cl-Et', 'NO2-COOH', 'SO3H-COOH', 'H-Me', 'OMe-Et', 'CN-OMe', 'NO2-OH', 'COOH-Pr', 'Pr-NH2', 'OH-NH2', 'OPr-NHMe', 'Cl-H', 'F-I', 'Br-NH2', 'Et-F', 'OEt-COOH', 'CN-SO3H', 'HCO-Et', 'CN-Br', 'NH2-Br', 'I-H', 'COOH-I', 'Ph-Br', 'OH-CN', 'Et-NO2', 'OMe-OEt', 'I-NH2', 'CN-Pr', 'NHMe-OEt', 'H-Br', 'OPr-OH', 'OH-Cl', 'Et-NHMe', 'OEt-Cl', 'Br-Ph', 'HCO-F', 'Br-Cl', 'COOH-Cl', 'Pr-NO2', 'I-Me', 'Me-OMe', 'F-Me', 'NHMe-OMe', 'Br-F', 'SO3H-OPr', 'OEt-NO2', 'Et-Cl', 'NH2-Et', 'OH-F', 'NHMe-F', 'OPr-NH2', 'SO3H-Pr', 'HCO-OEt', 'Me-Pr', 'OEt-Pr', 'OH-I', 'Pr-NHMe', 'Ph-HCO', 'I-OEt', 'COOH-Ph', 'OEt', 'Cl-NO2', 'Me-Cl', 'OMe', 'Et-CN', 'OPr-Pr', 'NO2-Br', 'Ph-Pr', 'NO2-Cl', 'COOH-Et', 'CN-Cl', 'Me-H', 'Me-Ph', 'H-F', 'Et-OEt', 'NH2-I', 'SO3H-H', 'H', 'Br-I'],
    'functional_groups-1':['NH2', 'HCO', 'Et', 'NO2', 'NHMe', 'Cl', 'OPr', 'F', 'I', 'OEt', 'OH', 'Pr', 'OMe', ' ', 'Me', 'COOH', 'CN', 'SO3H', 'Ph', 'Br', 'H'], 
    'functional_groups-2':['NH2', 'HCO', 'Et', 'NO2', 'NHMe', 'Cl', 'OPr', 'F', 'I', 'OEt', 'OH', 'Pr', 'OMe', ' ', 'Me', 'COOH', 'CN', 'SO3H', 'Ph', 'Br', 'H'], 
    "topology":['acs', 'bcu', 'etb', 'fof', 'nbo', 'pcu', 'pts', 'rht', 'sra', 'tbo', 'the'],
}

# นิยามฟังก์ชั่นสำหรับแปลงค่า categorical value เป็น index
def replace_cats(type, df):
  return df.replace(REPLACE_CATS[type], list(range(len(REPLACE_CATS[type])))).astype('int64')

# นิยามฟังก์ชั่นสำหรับ return ความยาวของ categorical values ที่มีทั้งหมดในคอลัมน์
def len_cats(type):
  return len(REPLACE_CATS[type])

# นิยามฟังก์ชั่นสำหรับการหารค่า metal linker, organic linker, functional groups ด้วย unit cell volume
def update_data(data):
    for e in ['n_ml', 'n_ml_m_n', '_n_ol1', '_n_ol2', '_n_fg1', '_n_fg2']:
        data[e] = data[e] / data['volume [A^3]']
    return data

In [ ]:
# คอลัมน์สำหรับ catagorical values
cat_cols = [ 'topology', 'functional_groups-1', 'functional_groups-2', 'metal_linker', 'organic_linker1', 'organic_linker2']

# คอลัมน์สำหรับ continuous values
cont_cols = [    
    'NUM_ATOMS', # จำนวนอะตอมรวม
    'Br', 'F', 'Cr', 'Cu', 'N', 'H', 'Zn', 'O', 'P', 'V', 'S', 'Ba', 'C', 'I', 'Ni', 'Cl', # จำนวนอะตอมธาตุ / จำนวนอะตอมรวม
    'sc', 
    'n_ml', # สัดส่วน metal linker / unit cell volume
    'n_m', 'n_ml_m_n', 
    
    # เป็นค่า Radial Distribution RDF-R-i โดย R คือ Radius และ i = 1 คือ electronegativity, 2 คือ polarization, 3 คือ van_der_waals volume
    'RDF-2-1', 'RDF-2-2', 'RDF-2-3', 
    'RDF-2.2-1', 'RDF-2.2-2', 'RDF-2.2-3', 
    'RDF-2.5-1', 'RDF-2.5-2', 'RDF-2.5-3', 
    'RDF-2.8-1', 'RDF-2.8-2', 'RDF-2.8-3',              
    'RDF-3-1', 'RDF-3-2', 'RDF-3-3', 
    'RDF-3.2-1', 'RDF-3.2-2', 'RDF-3.2-3',              
    'RDF-3.5-1', 'RDF-3.5-2', 'RDF-3.5-3', 
    'RDF-3.8-1', 'RDF-3.8-2', 'RDF-3.8-3',             
    'RDF-4-1', 'RDF-4-2', 'RDF-4-3',
    'RDF-4.5-1', 'RDF-4.5-2', 'RDF-4.5-3',
    'RDF-5-1', 'RDF-5-2', 'RDF-5-3', 
    'RDF-5.5-1', 'RDF-5.5-2', 'RDF-5.5-3',
    'RDF-6-1', 'RDF-6-2', 'RDF-6-3',
    'RDF-6.5-1', 'RDF-6.5-2', 'RDF-6.5-3', 
    'RDF-7-1', 'RDF-7-2', 'RDF-7-3',
    'RDF-7.5-1', 'RDF-7.5-2', 'RDF-7.5-3',
    'RDF-8-1', 'RDF-8-2', 'RDF-8-3',
    'RDF-8.5-1', 'RDF-8.5-2', 'RDF-8.5-3',
    'RDF-9-1', 'RDF-9-2', 'RDF-9-3', 
    'RDF-9.5-1', 'RDF-9.5-2', 'RDF-9.5-3', 
    'RDF-10-1', 'RDF-10-2', 'RDF-10-3',
    'RDF-10.5-1', 'RDF-10.5-2', 'RDF-10.5-3',
    'RDF-11-1', 'RDF-11-2', 'RDF-11-3',   
    'RDF-11.5-1', 'RDF-11.5-2', 'RDF-11.5-3',             
    'RDF-12-1', 'RDF-12-2', 'RDF-12-3', 
    'RDF-12.5-1', 'RDF-12.5-2', 'RDF-12.5-3',
    'RDF-13-1', 'RDF-13-2', 'RDF-13-3',
    'RDF-13.5-1', 'RDF-13.5-2', 'RDF-13.5-3',
    'RDF-14-1', 'RDF-14-2', 'RDF-14-3', 
    'RDF-14.5-1', 'RDF-14.5-2', 'RDF-14.5-3',
    'RDF-15-1', 'RDF-15-2', 'RDF-15-3', 
    'RDF-15.5-1', 'RDF-15.5-2', 'RDF-15.5-3',
    'RDF-16-1', 'RDF-16-2', 'RDF-16-3', 
    'RDF-16.5-1', 'RDF-16.5-2', 'RDF-16.5-3',
    'RDF-17-1', 'RDF-17-2', 'RDF-17-3',
    'RDF-17.5-1', 'RDF-17.5-2', 'RDF-17.5-3',
    'RDF-18-1', 'RDF-18-2', 'RDF-18-3', 
    'RDF-18.5-1', 'RDF-18.5-2', 'RDF-18.5-3',
    'RDF-19-1', 'RDF-19-2', 'RDF-19-3',
    'RDF-19.5-1', 'RDF-19.5-2', 'RDF-19.5-3',
    'RDF-20-1', 'RDF-20-2', 'RDF-20-3', 
    'RDF-20.5-1', 'RDF-20.5-2', 'RDF-20.5-3',
    'RDF-21-1', 'RDF-21-2', 'RDF-21-3',
    'RDF-21.5-1', 'RDF-21.5-2', 'RDF-21.5-3',
    'RDF-22-1', 'RDF-22-2', 'RDF-22-3', 
    'RDF-22.5-1', 'RDF-22.5-2', 'RDF-22.5-3',
    'RDF-23-1', 'RDF-23-2', 'RDF-23-3',
    'RDF-23.5-1', 'RDF-23.5-2', 'RDF-23.5-3',
    'RDF-24-1', 'RDF-24-2', 'RDF-24-3', 
    'RDF-24.5-1', 'RDF-24.5-2', 'RDF-24.5-3',
    'RDF-25-1', 'RDF-25-2', 'RDF-25-3', 
    'RDF-25.5-1', 'RDF-25.5-2', 'RDF-25.5-3',
    'RDF-26-1', 'RDF-26-2', 'RDF-26-3', 
    'RDF-26.5-1', 'RDF-26.5-2', 'RDF-26.5-3',
    'RDF-27-1', 'RDF-27-2', 'RDF-27-3',      
    'RDF-27.5-1', 'RDF-27.5-2', 'RDF-27.5-3',        
    'RDF-28-1', 'RDF-28-2', 'RDF-28-3', 
    'RDF-28.5-1', 'RDF-28.5-2', 'RDF-28.5-3',
    'RDF-29-1', 'RDF-29-2', 'RDF-29-3',      
    'RDF-29.5-1', 'RDF-29.5-2', 'RDF-29.5-3',                     
    'RDF-30-1', 'RDF-30-2', 'RDF-30-3', 
    
        'DEG_UNSAT', 'DEG_UNSAT_C', 'METAL_C', 'O_METAL', 'ELEC_FRAC', 'W_ELEC', 'N_C', 'O_C', 'N_O',
    'abc', # ค่าเฉลี่ยของ unit cell lengths
    'alpha-beta-gamma', # ค่าเฉลี่ยของ unit cell angles
    # ส่วนถัดไปเป็นฟีเจอร์จาก dataset โดย คำนวณหา density [g/cm^3] 
    'void_fraction', 'CO2/N2_selectivity', 'heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]', 'density [g/cm^3]', 
    'volume [A^3]', 'weight [u]', 'surface_area [m^2/g]', 'void_volume [cm^3/g]', 'density [g/cm^3]'
]

In [ ]:
# อ่าน train  dataset จาก csv
trainset = pd.read_csv('./trainset.csv')

# อ่าน test  dataset จาก csv
testset = pd.read_csv('./testset.csv')

# หารค่า metal linker, organic linker, functional groups ด้วย unitcell volume
trainset = update_data(trainset)
testset = update_data(testset)

# แปลงค่า categorical values ให้เป็น index
for col in cat_cols:
    trainset[col] = replace_cats(col, trainset[col])
    testset[col] = replace_cats(col, testset[col])

    
# สร้างชุดข้อมูลของ categorical values สำหรับ train dataset
X_train_cats = torch.tensor(np.stack([trainset[col].values for col in cat_cols], 1), dtype=torch.int64).to(device)

# สร้างชุดข้อมูลของ continous values สำหรับ train dataset
X_train_conts = torch.tensor(np.stack([trainset[col].values for col in cont_cols], 1), dtype=torch.float).to(device)

# ค่า label ของ train dataset
y_train = torch.tensor(trainset['CO2_working_capacity [mL/g]'].values, dtype=torch.float).reshape(-1,1).to(device)


# สร้างชุดข้อมูลของ categorical values สำหรับ test dataset
X_test_cats = torch.tensor(np.stack([testset[col].values for col in cat_cols], 1), dtype=torch.int64).to(device)

# สร้างชุดข้อมูลของ continous values สำหรับ test dataset
X_test_conts = torch.tensor(np.stack([testset[col].values for col in cont_cols], 1), dtype=torch.float).to(device)

# ค่า label ของ test dataset
y_test = torch.tensor(testset['CO2_working_capacity [mL/g]'].values, dtype=torch.float).reshape(-1,1).to(device)


# หาค่า category size ในแต่ละ columns ที่เป็นประเภท category
cat_szs = [len_cats(col) for col in cat_cols]

# หาค่า (input size, output size) ของ embedding layers 
# โดย output size อิงค่าจาก (input size + 1) // 2 ที่บวก 1 เพื่อให้ขั้นต่ำเป็น 1 โดยค่าสูงสุดจะไม่เกิน 50
emb_szs = [(size, min(50, (size+1)//2)) for size in cat_szs]

# สรุปรวม output size ของ embedding layer
print('Embedding size:', sum((nf for ni,nf in emb_szs)))

# สรุปขนาดของ trainset ทั้ง categorical values, continous values และ labels
print(X_train_cats.shape, X_train_conts.shape, y_train.shape)

# สรุปขนาดของ testset ทั้ง categorical values, continous values และ labels
print(X_test_cats.shape, X_test_conts.shape, y_test.shape)

# สรุปขนาดของ input size ก่อนจะเข้า MLPs
print('Total Size', sum((nf for ni,nf in emb_szs))+ X_train_conts.shape[1])

In [ ]:
# นิยาม Model
class MyModel(nn.Module):

    def __init__(self, emb_szs, n_cont, out_sz, layers, p=0.5):
        super().__init__()
    
        # สร้าง embedding layers โดยอิงจาก input size, output size ที่คำนวณไว้
        self.embeds = nn.ModuleList([nn.Embedding(ni, nf) for ni,nf in emb_szs])
        
        # นิยาม drop out
        self.emb_drop = nn.Dropout(p)
        
        # นิยาม batchnorm1d สำหรับ continous values        
        self.bn_cont = nn.BatchNorm1d(n_cont)
        
        # นิยาม output size จาก embedding layer
        n_emb = sum((nf for ni,nf in emb_szs))
        
        # นิยาม input size ก่อนเข้า MLPs
        n_in1 = n_emb + n_cont

        layerlist = []        
        for i in layers:
            layerlist.append(nn.Linear(n_in1, i)) 
            
            #ใช้ LeakyReLU เป็น activation function โดยมีค่าความชันด้านลบเป็น 0.2
            layerlist.append(nn.LeakyReLU(0.2, inplace=True))

            #ใส่ BatchNorm1d ให้ output            
            layerlist.append(nn.BatchNorm1d(i))
            
            # ใส่ drop out
            layerlist.append(nn.Dropout(p))
            
            n_in1 = i
        
        # กำหนด output layer
        layerlist.append(nn.Linear(layers[-1],out_sz))
        
        # นำ layer ที่สร้างไว้มาทำเป็น Sequential    
        self.layers = nn.Sequential(*layerlist)
    
    def forward(self, x_cat, x_cont):
        
        # นำ categorical values แต่ละคอลัมน์ มาผ่าน embedding layer แล้วเก็บไว้ในตัวแปร embeddings
        embeddings = []
        for i, e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
            
        # นำ categorical values ที่ผ่าน embedding layer มารวมกัน
        x = torch.cat(embeddings, 1)
        
        # นำไปผ่าน dropout
        x = self.emb_drop(x)
        
        # นำ continous values ไปผ่าน batchnorm1d
        x_cont = self.bn_cont(x_cont)

        # นำ categorical values และ continous values มาต่อกัน
        x = torch.cat([x, x_cont], 1)

        # นำ input ไปเข้า MLPs
        x = self.layers(x)

        # คืนค่า output
        return x

In [ ]:
# set seed เพื่อให้ initial weights เท่าเดิมสำหรับเปรียบเทียบ
torch.manual_seed(33)

# สร้าง data loader สำหรับ train ประกอบไปด้วย categorical values, countinous values และ label
train_loader = DataLoader(list(zip(X_train_cats, X_train_conts, y_train)), batch_size=1024, shuffle=True)

# นิยาม layers สำหรับ MLPs
layers = [4096, 2048, 1024, 512, 1024, 2048, 4096, 1024]

# สร้างตัวแปร model
model = MyModel(emb_szs, X_train_conts.shape[1], 1, layers, p=0.5)
model.to(device)

# นิยาม loss ที่ใช้งาน โดย L1Loss คือ MAE
l1_loss = nn.L1Loss()

# นิยาม initial learning rate
initial_lr = 0.01

# นิยาม optimizer ใช้ Adam optimizer ไม่ใส่ momentum และ weight decay
optimizer = torch.optim.Adam(model.parameters(), initial_lr)


# นิยามตัวแปรต่างๆ ที่ใช้งานในลูปการทำงานหลัก
epochs = 100000
train_losses = []
val_losses = []

patient = 200

epoch_min = 0

val_mape_min = 1e6
val_lmae_min = 1e6
err_mape = 0
err_lmae = 0

# เก็บโมเดลที่ดีที่สุด 10 อันดับไว้
MAX_MODELS = 10
model_mape_min = [{'mape':1e6, 'lmae':1e6, 'err':1e6, 'model':None} for i in range(MAX_MODELS)]
model_lmae_min = [{'mape':1e6, 'lmae':1e6, 'err':1e6, 'model':None} for i in range(MAX_MODELS)]


In [ ]:
# ลูปการทำงานหลัก

for i in range(epochs):
    i+=1

    # เข้าสู่ train mode
    model.train()
    
    batch_loss = []

    # อ่านข้อมูลมาทีละ batch จนครบ
    for x_cats, x_conts, y in train_loader:
        
        # predict
        y_pred = model(x_cats, x_conts)
        
        # คำนวณหา MAE
        MAE = l1_loss(y_pred, y)
        
        # คำนวณหา MAPE ใช้ค่าเฉลี่ยของ 2*abs(y_pred - y) / (abs(y_pred) + abs(y))
        MAPE = 2*(torch.abs(y_pred - y)/(torch.abs(y_pred)+torch.abs(y))).mean()
        
        # คำนวณหา train loss ใช้ log(MAE + 5*MAPE)
        train_loss = torch.log(MAE+ 5*MAPE)

        # update weight
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()      

        # เก็บ record LMAE ของ train ประจำ batch
        batch_loss.append(torch.log10(MAE).item())

    # หาค่า LMAE ของ train ประจำ epoch
    train_losses.append(np.mean(batch_loss))

    # เข้าสู่ eval mode    
    model.eval()
    with torch.no_grad():
        
        # predict        
        y_val = model(X_test_cats, X_test_conts)      

        # คำนวณหา val loss ซึ่งเป็น LMAE
        val_lmae = torch.log10(l1_loss(y_val, y_test))
        
        # เก็บค่า val lmae ประจำ epoch
        val_losses.append(val_lmae.item())
        
        # คำนวณหา val loss แบบ MAPE        
        val_mape = 2*(torch.abs(y_val - y_test)/(torch.abs(y_val)+torch.abs(y_test))).mean()

        if val_lmae < val_lmae_min:# เลือก model ที่มี val lmae ต่ำกว่า val lmae ที่สูงที่สุดที่เก็บไว้เป็นคำตอบ
            err_lmae = 100*torch.abs((y_val-y_test)/y_test).mean().item() # คำนวณหา percentage error
            max_model = ([m['lmae'] for m in model_lmae_min]) # เลือกโมเดลที่มีค่า lmae สูงสุดมาอ้างอิง
            # ปรับค่าโมเดลที่เลือกให้เป็นโมเดลใหม่ที่มี val lmae ต่ำกว่า
            model_lmae_min[np.argmax(max_model)] = {'mape':val_mape, 'lmae':val_lmae, 'err':err_lmae, 'model':model.state_dict()}
            max_model = ([m['lmae'] for m in model_lmae_min]) # เลือกโมเดลที่มีค่า lmae สูงสุดมาอ้างอิง
            # อัพเดทค่าอ้างอิงใหม่ โดยเป็นค่า 
            val_lmae_min = np.max(max_model)
            # เก็บค่า epoch ที่พบตัว val lmae ไว้ เพื่อใช้หยุดก่อนกำหนด            
            epoch_min = i       
            
            # สร้างตัวแปร y true และ y fake เพื่อใช้สร้างแผนภาพ
            y_t = y_test.cpu().numpy()
            y_f = y_val.cpu().numpy() 
            
        elif val_mape < val_mape_min:# เลือก model ที่มี val mape ต่ำกว่า val mape ที่สูงที่สุดที่เก็บไว้เป็นคำตอบ
            err_mape = 100*torch.abs((y_val-y_test)/y_test).mean().item() # คำนวณหา percentage error
            max_model = ([m['mape'] for m in model_mape_min]) # เลือกโมเดลที่มีค่า mape สูงสุดมาอ้างอิง
            # ปรับค่าโมเดลที่เลือกให้เป็นโมเดลใหม่ val mape ต่ำกว่า
            model_mape_min[np.argmax(max_model)] = {'mape':val_mape, 'lmae':val_lmae, 'err':err_mape, 'model':model.state_dict()}
            max_model = ([m['mape'] for m in model_mape_min]) # เลือกโมเดลที่มีค่า mape สูงสุดมาอ้างอิง
            # อัพเดทค่าอ้างอิงใหม่ โดยเป็นค่า 
            val_mape_min = np.max(max_model)
            # เก็บค่า epoch ที่พบตัว val mape ไว้ เพื่อใช้หยุดก่อนกำหนด
            epoch_min = i

    # ทุกๆ 10 epoch จะอัพเดทกราฟเพื่อให้เห็น loss และการกระจายตัวระหว่าง y_predict & y_true ของ validation
    if i%10 == 0:

        clear_output(wait=True)

        fig = plt.figure(figsize=(25, 10))

        ax1 = fig.add_subplot(1, 2, 1)
        color = 'tab:red'
        ax1.set_xlabel('epochs')
        ax1.set_ylabel('train-losses', color=color)        
        ax1.plot(range(1, len(train_losses)+1), [i for i in train_losses], color=color)
        ax1.tick_params(axis='y', labelcolor=color)
        ax2 = ax1.twinx()
        color = 'tab:blue'
        ax2.set_xlabel('epochs')
        ax2.set_ylabel('train-losses', color=color)            
        ax2.plot(range(1, len(val_losses)+1), [i for i in val_losses], color=color)
        ax2.tick_params(axis='y', labelcolor=color)

        fig.tight_layout()

        ax1 = fig.add_subplot(1, 2, 2, projection='scatter_density')
        density = ax1.scatter_density(y_t, y_f, cmap=white_viridis)
        fig.colorbar(density, label='Number of points per pixel')
        ax1.plot([-50.0,800.0], [-50.0,800.0], "--", color = "Black")
        ax1.set_ylabel('Predicted WC')
        ax1.set_xlabel('Real data WC')

        plt.show()

        # พิมพ์รายละเอียด
        print(f'epoch: {i:3}  train_loss: {train_losses[-1]:10.8f} val_loss: {val_losses[-1]:10.8f}  lr:{lr:10.8f}  min_lmae: {val_lmae_min:10.8f} err_lmae: {err_lmae:10.8f}% min_mape: {val_mape_min:10.8f} err_mape: {err_mape:10.8f}% epoch_min: {epoch_min:3}')

    # กรณีไม่พบ val_lmae min แล้วภายใน patient epochs ให้จบ
    if i - epoch_min > patient:
        print("-----EarlyStop-----")
        break

    # decay learning rate โดยจะลด 5% ทุกๆ 10 epochs
    lr = initial_lr * (0.95 ** (i // 10))

    # update learning rate ให้ optimizer
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

print(f'epoch: {i:3}  train_loss: {train_losses[-1]:10.8f} val_loss: {val_losses[-1]:10.8f}  lr:{lr:10.8f}  min_lmae: {val_lmae_min:10.8f} err_lmae: {err_lmae:10.8f}% min_mape: {val_mape_min:10.8f} err_mape: {err_mape:10.8f}% epoch_min: {epoch_min:3}')

In [ ]:
# สร้างโมเดลเพื่อสรุปค่า
model2 = MyModel(emb_szs, X_train_conts.shape[1], 1, layers, p=0.5)
model2.to(device)

# กำหนดตัวแปร prediction สำหรับ train และ validation เตรียมไว้
y_pred = torch.zeros((X_train_cats.shape[0], 1)).to(device)
y_val = torch.zeros((X_test_cats.shape[0], 1)).to(device)
count = 0

# นำโมเดลที่บันทึกไว้ออกมาเฉลี่ยค่า
for mname, model_list in [('LMAE', model_lmae_min)]:
    print(mname)
    for i, mlist in enumerate(model_list):
        print(f"\t{i}: val_lmae = {mlist['lmae']:10.8f} | val_mpae = {mlist['mape']:10.8f} | err = {mlist['err']:10.8f}")
        
        model2.load_state_dict(mlist['model'])# โหลด weight ใส่โมเดล
        
        model2.eval() # evaluation mode
        
        with torch.no_grad():
            y_pred += model2(X_train_cats, X_train_conts)
            y_val += model2(X_test_cats, X_test_conts)
            count+=1

# เฉลี่ยคำตอบจาก model ที่เก็บไว้
y_pred /= count
y_val /= count

# คำนวณหาชุด LMAE loss
train_loss = torch.log10(l1_loss(y_pred, y_train))
val_loss = torch.log10(l1_loss(y_val, y_test))
err = torch.abs((y_val-y_test)/y_test).mean()

print(f'TRAIN-LMAE:{train_loss}   |   VAL-LMAE:{val_loss}   |   %ERR: {err*100}\n\n')

# แสดงแผนภาพการกระจายตัวระหว่าง y predict กับ y true ของ validation
fig = plt.figure(figsize=(12,10))
using_mpl_scatter_density(fig, y_test.cpu().numpy().tolist(), y_val.cpu().numpy().tolist()) #

In [ ]:
# ส่งคำตอบ

# อ่านไฟล์ test
data_test = pd.read_csv('test_fixed.csv')

# คำนวณ metal linker, organic linkers, functional groups ต่อ unit cell volume
data_test = update_data(data_test)

# เปลี่ยนจาก categorical values เป็น index
for col in cat_cols:
    data_test[col] = replace_cats(col, data_test[col])

# สร้างชุดข้อมูลของ categorical values สำหรับ test
data_test_cats = torch.tensor(np.stack([data_test[col].values for col in cat_cols], 1), dtype=torch.int64).to(device)

# สร้างชุดข้อมูลของ continous values สำหรับ test
data_test_conts = torch.tensor(np.stack([data_test[col].values for col in cont_cols], 1), dtype=torch.float).to(device)

# สร้างโมเดลเพื่อสรุปค่า
model2 = MyModel(emb_szs, X_train_conts.shape[1], 1, layers, p=0.5)
model2.to(device)

# เตรียมตัวแปรไว้เก็บค่า
pred = torch.zeros((data_test_cats.shape[0], 1)).to(device)
count = 0

# รวมค่าการทำนายจากทุกโมเดลที่บันทึกไว้
for mname, model_list in [('LMAE', model_lmae_min)]:
    print(mname)
    for i, mlist in enumerate(model_list):
        model2.load_state_dict(mlist['model'])
        model2.eval()
        with torch.no_grad():
            pred += model2(data_test_cats, data_test_conts)
            count+=1

# เฉลี่ยคำตอบ
pred /= count

#สร้างไฟล์คำตอบ
submission = pd.DataFrame({
    "id": [str(68614+i) for i in range(17000)],
    "CO2_working_capacity [mL/g]": pred.cpu().numpy().T[0]
})

# บันทึกไฟล์คำตอบ
submission.to_csv('./submission.csv', index=False)